## Global properties 1

In [65]:
using PyCall
using Distributions
using Statistics
using DataFrames

using Distances
using Random , Printf

rootdir = "/home/stephane/Science/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

import PyPlot , CSV

wdir    = "$rootdir/products"
votdir  = "$rootdir/products/votable"
ocdir   = "$rootdir/products/oc"

cd(wdir)

In [66]:
## compute global properties

function compute_globalparams(df, dfout)
    ram= median(df[:ra])
    decm= median(df[:dec])
    lm= median(df[:l])
    bm= median(df[:b])
    distm= median(df[:distance])
    Xm= median(df[:X]) 
    Ym= median(df[:Y])
    Zm= median(df[:Z])
    vbm= median(df[:vb])
    vlm= median(df[:vl])

    Xgalm, Ygalm , Zgalm= galXYZ(lm , bm , distm) 
    
    push!(dfout, (ram, decm, lm, bm, distm, Xm, Ym, Zm, Xgalm, Ygalm, Zgalm, vlm, vbm))
    return(0)
    
end

compute_globalparams (generic function with 2 methods)

In [67]:
## Compute the X,Y,Z galactic coordinates (centered on the Galactic Center)
##

function galXYZ(l,b,distance)
    Rgal= 8.34
    
    return(1, 1 , 1)
end

galXYZ (generic function with 1 method)

In [68]:
## Main loop
##

macro main(paramfile)
    let
        println("## Starting main loop..")
        spl= CSV.read(paramfile)
        println("## $paramfile read..") 
        s=size(spl)
        
        dfout= DataFrame(ram= Float64[], decm= Float64[], lm= Float64[], bm= Float64[], 
            distm= Float64[], Xm= Float64[], Ym= Float64[], Zm= Float64[], Xgalm= Float64[], 
            Ygalm= Float64[], Zgalm= Float64[], vlm= Float64[], vbm= Float64[])
        
        for i in 1:s[1]
            votname = spl[:votname][i]
            println("## Starting with $votname")
            
            ocfile= @sprintf("%s/%s-oc.csv",ocdir,votname[1:end-4])
            @printf("### oc file: %s \n",ocfile)
            
            df= CSV.read(ocfile,delim=";")
            compute_globalparams(df, dfout)
        end
        
        fileout= @sprintf("%s-SCglobal.csv",paramfile[1:end-4])
        CSV.write(fileout, dfout, delim=";")
        println("## $fileout written...")
    end
end

@main (macro with 1 method)

In [69]:
@main("finalSample_metric0.01.csv")

## Starting main loop..
## finalSample_metric0.01.csv read..
## Starting with Alessi 1-2.1deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 1-2.1deg-oc.csv 
## Starting with Alessi 12-2.4deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 12-2.4deg-oc.csv 
## Starting with Alessi 13-15.9deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 13-15.9deg-oc.csv 
## Starting with Alessi 20-1.5deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 20-1.5deg-oc.csv 
## Starting with Alessi 21-1.6deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 21-1.6deg-oc.csv 
## Starting with Alessi 2-1.7deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 2-1.7deg-oc.csv 
## Starting with Alessi 3-2.2deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 3-2.2deg-oc.csv 
## Starting with Alessi 5-1.8deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Alessi 5-1.8deg-oc.csv 
## Starting with 

## Starting with ESO 128-16-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 128-16-1.0deg-oc.csv 
## Starting with ESO 130-06-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 130-06-1.0deg-oc.csv 
## Starting with ESO 130-08-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 130-08-1.0deg-oc.csv 
## Starting with ESO 132-14-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 132-14-1.0deg-oc.csv 
## Starting with ESO 260-06-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 260-06-1.0deg-oc.csv 
## Starting with ESO 261-03-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 261-03-1.0deg-oc.csv 
## Starting with ESO 311-14-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 311-14-1.0deg-oc.csv 
## Starting with ESO 313-12-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/ESO 313-12-1.0deg-oc.csv 
## Starting with ESO 332-20-1.0deg.vot
### oc file: /hom

### oc file: /home/stephane/Science/GAIA/products/oc/Haffner 13-1.6deg-oc.csv 
## Starting with Haffner 13-1.6deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Haffner 13-1.6deg-oc.csv 
## Starting with Harvard 16-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Harvard 16-1.0deg-oc.csv 
## Starting with Harvard 16-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Harvard 16-1.0deg-oc.csv 
## Starting with Harvard 16-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Harvard 16-1.0deg-oc.csv 
## Starting with Harvard 16-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Harvard 16-1.0deg-oc.csv 
## Starting with Hogg 10-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Hogg 10-1.0deg-oc.csv 
## Starting with Hogg 10-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Hogg 10-1.0deg-oc.csv 
## Starting with Hogg 10-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Hogg 10-1.0deg-oc.

## Starting with NGC 2287-2.2deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2287-2.2deg-oc.csv 
## Starting with NGC 2301-1.3deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2301-1.3deg-oc.csv 
## Starting with NGC 2302-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2302-1.0deg-oc.csv 
## Starting with NGC 2306-1.1deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2306-1.1deg-oc.csv 
## Starting with NGC 2318-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2318-1.0deg-oc.csv 
## Starting with NGC 2323-2.3deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2323-2.3deg-oc.csv 
## Starting with NGC 2335-1.6deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2335-1.6deg-oc.csv 
## Starting with NGC 2343-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 2343-1.0deg-oc.csv 
## Starting with NGC 2349-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/o

## Starting with NGC 752-7.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 752-7.0deg-oc.csv 
## Starting with NGC 7654-1.6deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 7654-1.6deg-oc.csv 
## Starting with NGC 7762-1.1deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 7762-1.1deg-oc.csv 
## Starting with NGC 7789-2.1deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 7789-2.1deg-oc.csv 
## Starting with NGC 886-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 886-1.0deg-oc.csv 
## Starting with NGC 957-1.4deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/NGC 957-1.4deg-oc.csv 
## Starting with Patchick 75-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Patchick 75-1.0deg-oc.csv 
## Starting with Per OB2-4.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc/Per OB2-4.0deg-oc.csv 
## Starting with Pismis 14-1.0deg.vot
### oc file: /home/stephane/Science/GAIA/products/oc